# OBJECT DETECTION USING TRAINED MODEL

* Load pipeline config, build object detection model, and restore chekpoint

In [1]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

# pipeline configuration
model_pipeline_config_file = os.path.join('models/pipeline.config')

# load pipeline config and build model
configs = config_util.get_configs_from_pipeline_file(model_pipeline_config_file)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
model_checkpoint = os.path.join('models/ckpt-3') ## todo: dynamically find last checkpoint
ckpt.restore(model_checkpoint).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

2021-08-27 12:34:16.755753: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-27 12:34:16.759461: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-27 12:34:16.759686: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-27 12:34:16.760079: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

InternalError: CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory

## 1. OBJECT DETECTION FOR IMAGES

In [ ]:
import os
import cv2 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

# file with class lables 
class_labels = os.path.join('annotations', 'annotations.pbtxt')
category_index = label_map_util.create_category_index_from_labelmap(class_labels)

# test images
test_image_1 = os.path.join('resources/images/test/5_scene.png')
test_image_2 = os.path.join('resources/images/test/10_scene.png')
test_image_3 = os.path.join('resources/images/test/15_scene.png')
test_image_4 = os.path.join('resources/images/test/20_scene.png')
test_image_5 = os.path.join('resources/images/test/25_scene.png')

img = cv2.imread(test_image_1)
image_np = np.array(img)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=5,
            min_score_thresh=.8,
            agnostic_mode=False)

plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
plt.show()

## 2. WEBCAM VIDEO STREAM

In [8]:
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while cap.isOpened(): 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.8,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break

2021-08-27 12:28:16.013530: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-08-27 12:28:16.013592: W tensorflow/core/framework/op_kernel.cc:1692] OP_REQUIRES failed at conv_ops.cc:1276 : Not found: No algorithm worked!


NotFoundError: 2 root error(s) found.
  (0) Not found:  No algorithm worked!
	 [[node ssd_mobile_net_v2fpn_keras_feature_extractor/model/Conv1/Conv2D (defined at home/everett/Repositories/research/projects/test/TFOD-model-training/model-training/tfod-venv/lib/python3.8/site-packages/object_detection/models/ssd_mobilenet_v2_fpn_keras_feature_extractor.py:219) ]]
	 [[Postprocessor/BatchMultiClassNonMaxSuppression/MultiClassNonMaxSuppression/Reshape/_4]]
  (1) Not found:  No algorithm worked!
	 [[node ssd_mobile_net_v2fpn_keras_feature_extractor/model/Conv1/Conv2D (defined at home/everett/Repositories/research/projects/test/TFOD-model-training/model-training/tfod-venv/lib/python3.8/site-packages/object_detection/models/ssd_mobilenet_v2_fpn_keras_feature_extractor.py:219) ]]
0 successful operations.
0 derived errors ignored. [Op:__inference_detect_fn_14805]

Errors may have originated from an input operation.
Input Source operations connected to node ssd_mobile_net_v2fpn_keras_feature_extr